# Deep Learning
## Assignment 2
Previously in 1_notmnist.ipynb, we created a pickle with formatted datasets for training, development and testing on the notMNIST dataset.

The goal of this assignment is to progressively train deeper and more accurate models using TensorFlow.

In [2]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle
from six.moves import range

First reload the data we generated in 1_notmnist.ipynb.

In [3]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a shape that's more adapted to the models we're going to train:
* data as a flat matrix,
* labels as float 1-hot encodings.

In [4]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 0 to [1.0, 0.0, 0.0 ...], 1 to [0.0, 1.0, 0.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


We're first going to train a multinomial logistic regression using simple gradient descent.

TensorFlow works like this:
* First you describe the computation that you want to see performed: what the inputs, the variables, and the operations look like. These get created as nodes over a computation graph. This description is all contained within the block below:
<br/><br/><pre><code>with graph.as_default():
...</code></pre>

    
    
* Then you can run the operations on this graph as many times as you want by calling session.run(), providing it outputs to fetch from the graph that get returned. This runtime operation is all contained in the block below:
<br/><br/><pre><code>with tf.Session(graph=graph) as session:
...</code></pre>

Let's load all the data into TensorFlow and build the computation graph corresponding to our training:

In [6]:
# With gradient descent training, even this much data is prohibitive.
# Subset the training data for faster turnaround.
train_subset = 10000

graph = tf.Graph()
with graph.as_default():

  # Input data.
  # Load the training, validation and test data into constants that are
  # attached to the graph.
  tf_train_dataset = tf.constant(train_dataset[:train_subset, :])
  tf_train_labels = tf.constant(train_labels[:train_subset])
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  # These are the parameters that we are going to be training. The weight
  # matrix will be initialized using random values following a (truncated)
  # normal distribution. The biases get initialized to zero.
  weights = tf.Variable(
    tf.truncated_normal([image_size * image_size, num_labels]))
  biases = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  # We multiply the inputs with the weight matrix, and add biases. We compute
  # the softmax and cross-entropy (it's one operation in TensorFlow, because
  # it's very common, and it can be optimized). We take the average of this
  # cross-entropy across all training examples: that's our loss.
  logits = tf.matmul(tf_train_dataset, weights) + biases
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
  
  # Optimizer.
  # We are going to find the minimum of this loss using gradient descent.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  # These are not part of training, but merely here so that we can report
  # accuracy figures as we train.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(tf.matmul(tf_valid_dataset, weights) + biases)
  test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)

Let's run this computation and iterate:

In [7]:
num_steps = 801

def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

with tf.Session(graph=graph) as session:
  # This is a one-time operation which ensures the parameters get initialized as
  # we described in the graph: random weights for the matrix, zeros for the
  # biases. 
  tf.initialize_all_variables().run()
  print('Initialized')
  for step in range(num_steps):
    # Run the computations. We tell .run() that we want to run the optimizer,
    # and get the loss value and the training predictions returned as numpy
    # arrays.
    _, l, predictions = session.run([optimizer, loss, train_prediction])
    if (step % 100 == 0):
      print('Loss at step %d: %f' % (step, l))
      print('Training accuracy: %.1f%%' % accuracy(
        predictions, train_labels[:train_subset, :]))
      # Calling .eval() on valid_prediction is basically like calling run(), but
      # just to get that one numpy array. Note that it recomputes all its graph
      # dependencies.
      print('Validation accuracy: %.1f%%\n' % accuracy(
        valid_prediction.eval(), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Loss at step 0: 16.539206
Training accuracy: 7.5%
Validation accuracy: 12.1%

Loss at step 100: 2.318529
Training accuracy: 71.2%
Validation accuracy: 70.7%

Loss at step 200: 1.875965
Training accuracy: 74.4%
Validation accuracy: 73.2%

Loss at step 300: 1.636832
Training accuracy: 75.6%
Validation accuracy: 74.0%

Loss at step 400: 1.472797
Training accuracy: 76.5%
Validation accuracy: 74.4%

Loss at step 500: 1.348928
Training accuracy: 77.3%
Validation accuracy: 74.6%

Loss at step 600: 1.250696
Training accuracy: 77.9%
Validation accuracy: 74.5%

Loss at step 700: 1.170672
Training accuracy: 78.3%
Validation accuracy: 74.8%

Loss at step 800: 1.104077
Training accuracy: 78.9%
Validation accuracy: 74.8%

Test accuracy: 83.0%


Let's now switch to stochastic gradient descent training instead, which is much faster.

The graph will be similar, except that instead of holding all the training data into a constant node, we create a Placeholder node which will be fed actual data at every call of session.run().

In [8]:
batch_size = 128

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights = tf.Variable(
    tf.truncated_normal([image_size * image_size, num_labels]))
  biases = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  logits = tf.matmul(tf_train_dataset, weights) + biases
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(tf.matmul(tf_valid_dataset, weights) + biases)
  test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)

Let's run it:

In [9]:
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%\n" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 20.271530
Minibatch accuracy: 10.2%
Validation accuracy: 16.4%

Minibatch loss at step 500: 1.334523
Minibatch accuracy: 75.0%
Validation accuracy: 75.2%

Minibatch loss at step 1000: 1.600177
Minibatch accuracy: 78.1%
Validation accuracy: 76.3%

Minibatch loss at step 1500: 0.778966
Minibatch accuracy: 83.6%
Validation accuracy: 76.4%

Minibatch loss at step 2000: 0.854264
Minibatch accuracy: 82.8%
Validation accuracy: 77.5%

Minibatch loss at step 2500: 0.937812
Minibatch accuracy: 72.7%
Validation accuracy: 78.0%

Minibatch loss at step 3000: 1.007037
Minibatch accuracy: 79.7%
Validation accuracy: 78.7%

Test accuracy: 86.1%


## Problem
Turn the logistic regression example with SGD into a 1-hidden layer neural network with rectified linear units nn.relu() and 1024 hidden nodes. This model should improve your validation / test accuracy.


In [5]:
batch_size = 128
H = 1024 # No. of hidden units

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights1 = tf.Variable(
    tf.truncated_normal([image_size * image_size, H]))
  biases1 = tf.Variable(tf.zeros([H]))
  weights2 = tf.Variable(
    tf.truncated_normal([H, num_labels]))
  biases2 = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  hidden1 = tf.nn.relu(tf.matmul(tf_train_dataset, weights1) + biases1)
  print(hidden1)
  logits = tf.matmul(hidden1, weights2) + biases2
  print(logits)
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
  print(loss)
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(tf.matmul(
                                   tf.nn.relu(tf.matmul(tf_valid_dataset, weights1) + biases1), weights2) + biases2)
  test_prediction = tf.nn.softmax(tf.matmul(
                                   tf.nn.relu(tf.matmul(tf_test_dataset, weights1) + biases1), weights2) + biases2)

Tensor("Relu:0", shape=TensorShape([Dimension(128), Dimension(1024)]), dtype=float32)
Tensor("add_1:0", shape=TensorShape([Dimension(128), Dimension(10)]), dtype=float32)
Tensor("Mean:0", shape=TensorShape([]), dtype=float32)


Let's run it !

In [11]:
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%\n" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 456.452698
Minibatch accuracy: 5.5%
Validation accuracy: 30.7%

Minibatch loss at step 500: 23.754307
Minibatch accuracy: 80.5%
Validation accuracy: 80.2%

Minibatch loss at step 1000: 10.889432
Minibatch accuracy: 78.9%
Validation accuracy: 80.7%

Minibatch loss at step 1500: 4.225248
Minibatch accuracy: 87.5%
Validation accuracy: 81.5%

Minibatch loss at step 2000: 1.746160
Minibatch accuracy: 89.8%
Validation accuracy: 80.5%

Minibatch loss at step 2500: 2.624768
Minibatch accuracy: 83.6%
Validation accuracy: 82.0%

Minibatch loss at step 3000: 2.049767
Minibatch accuracy: 82.0%
Validation accuracy: 82.3%

Test accuracy: 89.6%


## Deep Neural Network - 2 Hidden Layers
Lets make the model more Deep ! Lets add one more hidden layer !

**** 
There is some problem with TF regarding due to which we are getting NaN values...So, this part was not working initially

Solution to the problem:
Try lowering the learning rate to say 0.03. Now SGD will take time to converge. To reduce convergence time, add std deviation to the randomly generated weights. The value of std dev should be sqrt(2 / length_of_flattened_weights_vector). 
For eg:
weights1 = tf.Variable(tf.truncated_normal([image_size * image_size, H], 
stddev=sqrt(2.0/(image_size * image_size * H))))
Do this for all the weights. This will not allow the weights to diverge.
Using standard deviation while randomly assigning weights
which prevents them from diverging which in-turn doesn't cause NaN values to occur in ReLU vectors(activations of hidden layers)

In [30]:
batch_size = 128
H = 1024 # No. of hidden units

from math import sqrt

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables. (used stddev=sqrt(2/(length_of_flattened_matrix) --> This avoids diverges of weights 
  #                                                                  which doesn't cause NaN values 
  #                                                                   to occure in ReLU vectors)) 
  weights1 = tf.Variable(tf.truncated_normal([image_size * image_size, H], 
                                             stddev=sqrt(2.0/(image_size * image_size * H))))
  biases1 = tf.Variable(tf.zeros([H]))
  weights2 = tf.Variable(tf.truncated_normal([H, H],
                                            stddev=sqrt(2.0/(H * H))))
  biases2 = tf.Variable(tf.zeros([H]))
  weights3 = tf.Variable(
    tf.truncated_normal([H, num_labels]))
  biases3 = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  hidden1 = tf.nn.relu(tf.matmul(tf_train_dataset, weights1) + biases1)
  hidden2 = tf.nn.relu(tf.matmul(hidden1, weights2) + biases2)
  logits = tf.matmul(hidden2, weights3) + biases3
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.03).minimize(loss) # used alpha=0.03
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  
  h1_valid = tf.nn.relu(tf.matmul(tf_valid_dataset, weights1) + biases1)
  h2_valid = tf.nn.relu(tf.matmul(h1_valid, weights2) + biases2)
  valid_logits = tf.matmul(h2_valid, weights3) + biases3
  valid_prediction = tf.nn.softmax(valid_logits)
    
  h1_test = tf.nn.relu(tf.matmul(tf_test_dataset, weights1) + biases1)
  h2_test = tf.nn.relu(tf.matmul(h1_test, weights2) + biases2)
  test_logits = tf.matmul(h2_test, weights3) + biases3
  test_prediction = tf.nn.softmax(test_logits)

In [29]:
num_steps = 5001 # try 4001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%\n" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 2.302948
Minibatch accuracy: 10.2%
Validation accuracy: 15.8%

Minibatch loss at step 500: 0.347478
Minibatch accuracy: 89.8%
Validation accuracy: 85.1%

Minibatch loss at step 1000: 0.505625
Minibatch accuracy: 84.4%
Validation accuracy: 86.6%

Minibatch loss at step 1500: 0.268360
Minibatch accuracy: 92.2%
Validation accuracy: 88.1%

Minibatch loss at step 2000: 0.247440
Minibatch accuracy: 94.5%
Validation accuracy: 88.3%

Minibatch loss at step 2500: 0.315634
Minibatch accuracy: 89.8%
Validation accuracy: 88.5%

Minibatch loss at step 3000: 0.324441
Minibatch accuracy: 89.1%
Validation accuracy: 88.9%

Minibatch loss at step 3500: 0.378142
Minibatch accuracy: 89.1%
Validation accuracy: 89.4%

Minibatch loss at step 4000: 0.242072
Minibatch accuracy: 92.2%
Validation accuracy: 89.4%

Minibatch loss at step 4500: 0.259130
Minibatch accuracy: 91.4%
Validation accuracy: 89.3%

Minibatch loss at step 5000: 0.326131
Minibatch accuracy: 91.4%
Validati